In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd

from keras import utils
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Dense, Reshape, Conv2D, AveragePooling2D, Flatten

## Load the data

In [ ]:
train = pd.read_csv('../input/train.csv')
labels = (train.iloc[:, 0].values).astype('int32')

X_train = (train.iloc[:, 1:].values).astype('float32')
X_test = (pd.read_csv('../input/test.csv').values).astype('float32')

## Explore the data

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train = utils.to_categorical(labels, num_classes=10)

In [ ]:
scale = np.max(X_train)
X_train /= scale
X_test /= scale

mean = np.std(X_train)
X_train -= mean
X_test -= mean

In [ ]:
split_ratio = 0.1
split_index = int(split_ratio * X_train.shape[0])
x_val = X_train[:split_index]
partial_x_train = X_train[split_index:]

y_val = y_train[:split_index]
partial_y_train = y_train[split_index:]

## Building the network

In [ ]:
kaynet = Sequential()

kaynet.add(Reshape(target_shape=(1, 28, 28), input_shape=(784,)))
kaynet.add(Conv2D(kernel_size=(3, 3), filters=32, padding="same", data_format="channels_first", kernel_initializer="uniform", use_bias=False))
kaynet.add(AveragePooling2D(pool_size=(2, 2), data_format="channels_first"))
kaynet.add(Conv2D(kernel_size=(3, 3), filters=32, padding="same", data_format="channels_first", kernel_initializer="uniform", use_bias=False))
kaynet.add(AveragePooling2D(pool_size=(2, 2), data_format="channels_first"))
kaynet.add(Flatten())
kaynet.add(Dense(units=1000, activation='relu'))
kaynet.add(Dense(units=100, activation='relu'))
kaynet.add(Dense(units=10, activation='sigmoid'))

In [ ]:
kaynet.compile(optimizer=RMSprop(lr=0.0005),
               loss='categorical_crossentropy',
               metrics=['accuracy'])

In [ ]:
history = kaynet.fit(partial_x_train,
                    partial_y_train,
                    epochs=60,
                    batch_size=64,
                    validation_data=(x_val, y_val))

In [ ]:
history_dict = history.history
history_dict.keys()

## Plotting

### Plotting the training and validation loss

In [ ]:
import matplotlib.pyplot as plt

history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, 60)

plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

### Plotting the training and validation accuracy

In [ ]:
plt.clf()

acc_values = history_dict['binary_accuracy'] 
val_acc_values = history_dict['val_binary_accuracy']

plt.plot(epochs, acc_values, 'bo', label='Training acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

## Generating test predictions

In [ ]:
preds = kaynet.predict_classes(X_test)

def write_preds(preds, fname):
    pd.DataFrame({"ImageId": list(range(1,len(preds)+1)), "Label": preds}).to_csv(fname, index=False, header=True)

write_preds(preds, "kay-digit-recognizer.csv")